In [ ]:
import pandas as pd
import tushare as ts
import os
import time, datetime
import threadpool

def getDailyData():
    # create folder
    cwd = os.getcwd()
    today_date = time.strftime("%Y%m%d")
    target_dir = os.path.join("/home/samx/ramdisk/", today_date)
    try:
        os.makedirs(name=target_dir)
    except:
        print ('folder already existed')

    # clean up the index with numbers, and make date the default index
    def pre_process(df):
        df.set_index('date', inplace=True)
        # df.drop(df.columns[0], axis=1, inplace=True)
        return df

    def obtain_daily_data(sk_code, target_dir = ""):
        print ("Processing: "+sk_code)

        if target_dir!="":
            target_dir=target_dir+os.path.sep

        day_df = pre_process(ts.get_k_data(sk_code))
        csv_filename = target_dir + "day_" + sk_code + ".csv"
        day_df.to_csv(csv_filename)

        five_min_df = pre_process(ts.get_k_data(sk_code, ktype='5'))
        csv_filename = target_dir + "5min_" + sk_code + ".csv"
        five_min_df.to_csv(csv_filename)

        print (day_df.head(1))
        print (five_min_df.head(1))

    # obtain stock list
    stock_basics = ts.get_stock_basics()
    code_series = stock_basics.index

    # prepare arguments
    args_list=[]
    for code in code_series:
        args_list.append(([code, target_dir], None))

    # collect
    pool = threadpool.ThreadPool(30)
    requests = threadpool.makeRequests(obtain_daily_data, args_list)
    [pool.putRequest(req) for req in requests]
    pool.wait()

    print ('done')


while (True):
    now = datetime.datetime.now()
    today_end2 = datetime.datetime(now.year, now.month, now.day, 15, 1)
    tomorrow_end2 = today_end2 + datetime.timedelta(days=1)

    while (now-today_end2).days < 0: 
        time.sleep(3600)
    
    mark_begin = datetime.datetime.now()
    # execution--------------------------
    getDailyData()
    # execution--------------------------
    print(datetime.datetime.now() - mark_begin)
    
    while (now-tomorrow_end2).days < 0: 
        time.sleep(3600)

Processing: 603499
Processing: 002905
Processing: 300708Processing: 000786Processing: 002264Processing: 603367



Processing: 300705
Processing: 603527
Processing: 603110
Processing: 603103Processing: 300706Processing: 002902Processing: 603378Processing: 603797Processing: 603363





Processing: 603177
Processing: 600358
Processing: 300701
Processing: 300654
Processing: 300536
Processing: 002903
Processing: 603955
Processing: 603822
Processing: 002901
Processing: 002906
Processing: 300697
Processing: 603918
Processing: 300707
Processing: 603996
Processing: 603533
             open  close   high    low  volume    code
date                                                  
2017-09-29  16.96  20.35  20.35  16.96   163.0  002902            open  close  high   low  volume    code
date                                               
2017-09-26  9.25   11.1  11.1  9.25    89.0  300654
                   open  close   high    low  volume    code
date                                             

             open   close    high    low  volume    code
date                                                    
2016-12-13  12.35  14.816  14.816  12.35   351.0  603708
                   open  close   high    low  volume    code
date                                                        
2017-09-25 14:55  16.21  16.25  16.25  16.17  1311.0  603708
Processing: 002383
              open   close    high     low   volume    code
date                                                       
2014-05-21  15.702  16.698  16.698  15.702  14959.0  300298
                  open  close  high    low  volume    code
date                                                      
2017-09-25 14:55  21.2  21.16  21.2  21.13   318.0  300298
Processing: 300699
              open   close   high     low   volume    code
date                                                      
2014-12-22  17.312  15.737  17.36  15.561  57106.0  600452
                  open  close  high   low  volume    code
date            

             open  close   high    low    volume    code
date                                                    
2014-05-23  2.208  2.218  2.218  2.189  799089.0  000100
                  open  close  high  low   volume    code
date                                                     
2017-09-25 14:55   3.5   3.51  3.52  3.5  19782.0  000100
Processing: 000630
             open  close   high    low  volume    code
date                                                  
2014-07-28  3.266  3.327  3.343  3.266  7866.0  002537
             open  close   high    low   volume    code
date                                                   
2015-02-02  8.716  8.825  8.914  8.567  63230.0  002460             open  close   high    low  volume    code
date                                                  
2014-04-18  3.162  3.162  3.162  3.162  5480.0  002127

                   open  close   high    low  volume    code
date                                                        
2017-09-25 14:55

Processing: 002139              open   close    high     low    volume    code
date                                                        
2015-03-04  19.546  20.419  20.557  19.447  515196.0  000998

                  open  close   high   low  volume    code
date                                                      
2017-09-25 14:55  24.6  24.64  24.65  24.5  3464.0  000998
Processing: 000597
             open  close   high    low   volume    code
date                                                   
2015-02-25  6.053  6.013  6.082  5.984  47607.0  002042
                   open  close  high    low  volume    code
date                                                       
2017-09-25 14:55  10.99  10.98  11.0  10.98  2111.0  002042
Processing: 600338
             open  close   high    low  volume    code
date                                                  
2015-06-30  5.974  6.514  6.514  5.974    99.0  300481
                   open  close   high    low  volume    code
date     

            open  close   high    low   volume    code
date                                                  
2014-10-17  3.12  3.027  3.141  2.948  58956.0  002027
                   open  close   high    low  volume    code
date                                                        
2017-09-25 14:55  10.03  10.03  10.04  10.02  4671.0  002027
Processing: 300631
             open  close   high    low  volume    code
date                                                  
2017-01-12  7.803  9.362  9.362  7.803   149.0  002824              open  close    high     low    volume    code
date                                                       
2015-01-07  11.643  12.22  12.568  11.464  204589.0  600230

                   open  close   high   low  volume    code
date                                                       
2017-09-25 14:55  23.57  23.48  23.57  23.2  2535.0  002824                  open  close  high    low   volume    code
date                                             

...

             open   close    high    low   volume    code
date                                                     
2015-06-26  22.39  26.868  26.868  22.39  20680.0  601211
                   open  close   high    low   volume    code
date                                                         
2017-09-25 14:55  21.76  21.79  21.85  21.76  41052.0  601211
Processing: 601186
              open   close    high     low     volume    code
date                                                         
2015-03-04  12.844  12.632  12.911  12.536  1345609.0  601186
                   open  close   high   low   volume    code
date                                                        
2017-09-25 14:55  12.01   12.0  12.02  12.0  16460.0  601186
Processing: 601179
            open  close  high   low    volume    code
date                                                 
2015-03-02  4.35   4.56  4.56  4.34  219495.0  601558
                  open  close  high   low   volume    code
date        

              open   close    high     low    volume    code
date                                                        
2015-03-04  17.355  18.988  18.988  17.099  445790.0  601098
                   open  close   high    low  volume    code
date                                                        
2017-09-25 14:55  15.26  15.23  15.27  15.21  4406.0  601098
Processing: 601069
            open  close   high   low  volume    code
date                                                
2015-01-22  4.26  5.116  5.116  4.26  1285.0  601069
                  open  close   high    low  volume    code
date                                                       
2017-09-25 14:55  21.2   21.2  21.21  21.14  4239.0  601069
Processing: 601058
            open  close  high  low    volume    code
date                                                
2015-01-29  5.65   5.65  5.78  5.6  728052.0  601600
                  open  close  high   low    volume    code
date                                  

              open  close    high     low   volume    code
date                                                      
2014-12-02  10.623  10.76  10.869  10.623  27369.0  601137
                   open  close   high    low  volume    code
date                                                        
2017-09-25 14:55  12.16  12.17  12.19  12.16   756.0  601137
Processing: 600996
             open  close   high    low   volume    code
date                                                   
2015-02-27  9.035  9.059  9.203  9.011  57475.0  601126
                  open  close  high  low  volume    code
date                                                    
2017-09-25 14:55  8.93   8.93  8.93  8.9  1494.0  601126
Processing: 600995
            open  close  high  low    volume    code
date                                                
2015-02-09  7.95   8.01  8.08  7.9  310376.0  601118
                  open  close  high   low   volume    code
date                                         

              open   close    high     low     volume    code
date                                                         
2015-12-04  17.035  21.929  24.649  16.512  1767140.0  601155
                   open  close  high    low  volume    code
date                                                       
2017-09-25 14:55  18.17  18.26  18.3  18.16  6275.0  601155
Processing: 600963
             open  close  high    low  volume    code
date                                                 
2016-06-15  6.878   8.26  8.26  6.878   673.0  601127
                   open  close   high    low  volume    code
date                                                        
2017-09-25 14:55  21.01   21.0  21.05  20.95  2005.0  601127
Processing: 600962
              open   close   high     low    volume    code
date                                                       
2015-02-09  19.626  19.388  20.34  19.189  207238.0  600998
                   open  close   high    low  volume    code
date      

              open   close    high     low  volume    code
date                                                      
2016-08-09  10.571  12.685  12.685  10.571  1803.0  600977
                   open  close   high    low  volume    code
date                                                        
2017-09-25 14:55  17.12  17.11  17.12  17.08  2006.0  600977
Processing: 600888
             open  close   high   low   volume    code
date                                                  
2014-08-06  4.265  4.298  4.308  4.24  13106.0  600892
                  open  close   high    low  volume    code
date                                                       
2017-09-25 14:55  11.1  11.09  11.11  11.09   779.0  600892
Processing: 600887
              open   close    high     low   volume    code
date                                                       
2015-02-11  27.105  26.597  27.154  26.118  38515.0  600976
                   open  close   high    low  volume    code
date            

In [4]:
import datetime
now = datetime.datetime.now()
today_end2 = datetime.datetime(now.year, now.month, now.day, 15, 1)
tomorrow_end2 = today_end2 + datetime.timedelta(days=1)
today_end2, tomorrow_end2

(datetime.datetime(2017, 10, 16, 15, 1),
 datetime.datetime(2017, 10, 17, 15, 1))

In [4]:
getDailyData()

Processing: 603110Processing: 002903Processing: 002906Processing: 601086Processing: 603363Processing: 300425Processing: 300534Processing: 600499Processing: 603103Processing: 300705Processing: 603058Processing: 000560Processing: 603717Processing: 603568Processing: 300622Processing: 002846Processing: 300422Processing: 603603Processing: 603157


















Processing: 300665Processing: 002728Processing: 000791Processing: 600315Processing: 002843Processing: 300706Processing: 300536Processing: 002901Processing: 603955Processing: 300692Processing: 002499










             open  close   high    low  volume    code
date                                                  
2017-08-21  10.54  12.64  12.64  10.54    87.0  300692
                  open  close  high    low  volume    code
date                                                      
2017-09-22 14:55  60.8  59.57  60.8  58.29  8732.0  300692             open  close   high    low  volume    code
date                              

             open  close   high    low  volume    code
date                                                  
2017-01-20  7.915  8.733  8.733  7.915   144.0  002843
             open  close  high    low  volume    code
date                                                 
2017-02-17  8.073   9.69  9.69  8.073    73.0  603603                  open  close  high    low  volume    code
date                                                      
2017-09-22 14:55  25.0  24.95  25.0  24.91  1034.0  002843
             open  close   high    low  volume    code
date                                                  
2015-02-17  2.389  2.867  2.867  2.389    99.0  300422
Processing: 300650

                   open  close   high    low  volume    code
date                                                        
2017-09-22 14:55  18.21  18.32  18.32  18.21  1737.0  300422                   open  close   high    low  volume    code
date                                                        
2017-09-

Processing: 002010
             open  close   high    low  volume    code
date                                                  
2017-09-22  13.87  16.65  16.65  13.87   168.0  002901
             open  close   high    low  volume    code
date                                                  
2015-02-16  4.555  4.968  4.968  4.555   290.0  300425
                   open  close   high    low  volume    code
date                                                        
2017-09-22 14:55  16.65  16.65  16.65  16.65     0.0  002901
                   open  close  high    low  volume    code
date                                                       
2017-09-22 14:55  11.69   11.7  11.7  11.64  1011.0  300425
Processing: 002862
Processing: 300316
              open   close    high     low  volume    code
date                                                      
2017-01-05  18.531  20.223  20.223  18.531    84.0  300587
                   open  close   high    low  volume    code
date        

             open  close   high    low    volume    code
date                                                    
2015-01-30  8.644  8.601  8.767  8.573  123644.0  002456
                   open  close  high   low  volume    code
date                                                      
2017-09-22 14:55  20.43  20.47  20.5  20.4  6944.0  002456
Processing: 002826
             open   close    high     low   volume    code
date                                                      
2014-06-26  13.88  13.473  14.024  13.275  25836.0  300190
                   open  close   high    low  volume    code
date                                                        
2017-09-22 14:55  13.88  13.93  13.95  13.88   290.0  300190
Processing: 600654
              open  close    high     low    volume    code
date                                                       
2015-03-02  19.903  19.45  20.129  19.201  233115.0  600292
                   open  close   high    low  volume    code
date         

2015-02-06  11.897  11.504  11.99  11.317  73286.0  002229
              open   close    high     low  volume    code
date                                                      
2017-03-31  10.202  11.124  11.124  10.202    34.0  300635


                   open  close   high    low  volume    code
date                                                        
2017-09-22 14:55  20.66  20.63  20.68  20.61  1948.0  002036
                  open  close   high   low  volume    code
date                                                      
2017-09-22 14:55  48.6  48.74  48.97  48.6  1516.0  300635Processing: 300631                   open  close   high    low  volume    code
date                                                        
2017-09-20 14:55  16.41  16.41  16.41  16.41     2.0  002893


Processing: 002823Processing: 000065              open   close    high    low  volume    code
date                                                     
2014-12-26  14.853  14.713  14.858  14.39  9165.

...

Traceback (most recent call last):
  File "/home/samx/miniconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2442, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc (pandas/_libs/index.c:5280)
  File "pandas/_libs/index.pyx", line 154, in pandas._libs.index.IndexEngine.get_loc (pandas/_libs/index.c:5126)
  File "pandas/_libs/hashtable_class_helper.pxi", line 1210, in pandas._libs.hashtable.PyObjectHashTable.get_item (pandas/_libs/hashtable.c:20523)
  File "pandas/_libs/hashtable_class_helper.pxi", line 1218, in pandas._libs.hashtable.PyObjectHashTable.get_item (pandas/_libs/hashtable.c:20477)
KeyError: 'date'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/samx/miniconda3/lib/python3.6/site-packages/threadpool.py", line 158, in run
    result = request.callable(*request.args, **request.kwds)
  File "<ipython-input-2

Traceback (most recent call last):
  File "/home/samx/miniconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2442, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc (pandas/_libs/index.c:5280)
  File "pandas/_libs/index.pyx", line 154, in pandas._libs.index.IndexEngine.get_loc (pandas/_libs/index.c:5126)
  File "pandas/_libs/hashtable_class_helper.pxi", line 1210, in pandas._libs.hashtable.PyObjectHashTable.get_item (pandas/_libs/hashtable.c:20523)
  File "pandas/_libs/hashtable_class_helper.pxi", line 1218, in pandas._libs.hashtable.PyObjectHashTable.get_item (pandas/_libs/hashtable.c:20477)
KeyError: 'date'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/samx/miniconda3/lib/python3.6/site-packages/threadpool.py", line 158, in run
    result = request.callable(*request.args, **request.kwds)
  File "<ipython-input-2

              open  close   high     low   volume    code
date                                                     
2015-03-04  16.356  16.75  16.76  16.356  35452.0  600345
                   open  close   high   low  volume    code
date                                                       
2017-09-22 14:55  23.65  23.61  23.65  23.6  2842.0  600345
             open  close   high    low  volume    code
date                                                  
2016-11-04  6.605  7.926  7.926  6.605    53.0  300556
                   open  close   high   low  volume    code
date                                                       
2017-09-22 14:55  32.38   32.3  32.39  32.3   804.0  300556
            open  close  high   low    volume    code
date                                                 
2013-09-09  4.24   4.25  4.27  4.23  165036.0  000511
                  open  close  high   low   volume    code
date                                                      
2017-04-14 14:55  5.9

Traceback (most recent call last):
  File "/home/samx/miniconda3/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2442, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc (pandas/_libs/index.c:5280)
  File "pandas/_libs/index.pyx", line 154, in pandas._libs.index.IndexEngine.get_loc (pandas/_libs/index.c:5126)
  File "pandas/_libs/hashtable_class_helper.pxi", line 1210, in pandas._libs.hashtable.PyObjectHashTable.get_item (pandas/_libs/hashtable.c:20523)
  File "pandas/_libs/hashtable_class_helper.pxi", line 1218, in pandas._libs.hashtable.PyObjectHashTable.get_item (pandas/_libs/hashtable.c:20477)
KeyError: 'date'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/samx/miniconda3/lib/python3.6/site-packages/threadpool.py", line 158, in run
    result = request.callable(*request.args, **request.kwds)
  File "<ipython-input-2

              open   close    high    low   volume    code
date                                                      
2015-03-04  19.155  19.601  19.645  18.86  39919.0  300139
                   open  close   high    low  volume    code
date                                                        
2017-09-22 14:55  10.46  10.45  10.48  10.45  1657.0  300139
             open  close   high    low  volume    code
date                                                  
2017-05-04  10.11  10.11  10.11  10.11   149.0  603501
                  open  close   high    low   volume    code
date                                                        
2017-05-31 09:35  23.0  24.11  24.11  22.73  48793.0  603501
            open  close  high   low    volume    code
date                                                 
2014-02-17  4.15   4.21  4.27  4.09  174870.0  000950
                   open  close   high    low  volume    code
date                                                        
2017-04-